In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import re
import json
import gzip
import csv
import io
from polyglot.detect import Detector
import icu
from pprint import pprint

### Source: http://www.nltk.org/nltk_data/

### Filtering out everything except text

In [2]:
!ls ../Data/

negative_tweets_filtered.txt.gz
negative_tweets_ready.txt.gz
negative_tweets_ready_withstop.txt.gz
negative_tweets.txt.gz
positive_tweets_filtered.txt.gz
positive_tweets_ready.txt.gz
positive_tweets_ready_withstop.txt.gz
positive_tweets.txt.gz
tweets.20150430-223406_filtered.txt.gz
tweets.20150430-223406_ready.txt.gz
tweets.20150430-223406_ready_withstop.txt.gz
tweets.20150430-223406.txt.gz
twitter_samples.zip
World_twitterdata_part1.txt.gz
World_twitterdata_part2.txt.gz
World_twitterdata_part3.txt.gz
World_twitterdata_part4.txt.gz
World_twitterdata_part5.txt.gz
World_twitterdata_ready_part1.txt.gz
World_twitterdata_ready_part2.txt.gz
World_twitterdata_ready_part3.txt.gz
World_twitterdata_ready_part4.txt.gz
World_twitterdata_ready_part5.txt.gz
World_twitterdata_ready.txt.gz
World_twitterdata.txt.gz


In [3]:
def English_language_filtering( json_parsed ):
    if (json_parsed['lang'] == 'en'): # tweet is in English so nothing need to be done here
        return True
    else:
        return False  

In [4]:
%%time
filename = '../Data/negative_tweets.txt.gz'
file_out = gzip.open('../Data/negative_tweets_filtered.txt.gz', mode='wb')
i = 0
j = 0
with io.TextIOWrapper( io.BufferedReader(gzip.open(filename)) ) as file_in:
    for line in file_in: # read one line at a time from a compressed file
        json_parsed = json.loads(line) # parse only that one line to json
        lang_bool = English_language_filtering(json_parsed) # True: English, False: not English
        retweet_bool = 'retweeted_status' not in list(json_parsed.keys()) # True: not retweet, False: retweet
        if lang_bool == True and retweet_bool: # if conditions are met write to file
            file_out.write( json.dumps( {'text': json_parsed['text']} ).encode() ) # write dict to file
            file_out.write( '\n'.encode() ) # write a newline char to the end
            #print(i, json_parsed['text'], '\n')
            j += 1
        i += 1
file_in.close()
file_out.close()
print('Number of rows:', i, 'after filtering for language and retweet:', j)

Number of rows: 5000 after filtering for language and retweet: 5000
CPU times: user 327 ms, sys: 0 ns, total: 327 ms
Wall time: 329 ms


#### Check if file is correctly written

In [5]:
with io.TextIOWrapper( io.BufferedReader(gzip.open('../Data/negative_tweets_filtered.txt.gz')) ) as file_in:
    for line in file_in: # read one line at a time from a compressed file
        print(json.loads(line)) # parse only that one line to json

{'text': 'hopeless for tmr :('}
{'text': "Everything in the kids section of IKEA is so cute. Shame I'm nearly 19 in 2 months :("}
{'text': '@Hegelbon That heart sliding into the waste basket. :('}
{'text': '“@ketchBurning: I hate Japanese call him "bani" :( :(”\n\nMe too'}
{'text': 'Dang starting next week I have "work" :('}
{'text': "oh god, my babies' faces :( https://t.co/9fcwGvaki0"}
{'text': '@RileyMcDonough make me smile :(('}
{'text': '@f0ggstar @stuartthull work neighbour on motors. Asked why and he said hates the updates on search :( http://t.co/XvmTUikWln'}
{'text': 'why?:("@tahuodyy: sialan:( https://t.co/Hv1i0xcrL2"'}
{'text': 'Athabasca glacier was there in #1948 :-( #athabasca #glacier #jasper #jaspernationalpark #alberta #explorealberta #… http://t.co/dZZdqmf7Cz'}
{'text': "I have a really good m&amp;g idea but I'm never going to meet them :((("}
{'text': '@Rampageinthebox mare ivan :('}
{'text': '@SophiaMascardo happy trip, keep safe. see you soon :* :('}
{'text': "I'm 

{'text': '@sandwichlove_ I MISS them so much :('}
{'text': 'Going to sleep alone :('}
{'text': '@yongshwa huhu i know :-( thanks before satya! xx'}
{'text': 'My Aditya, mommy miss u sooo much :( .'}
{'text': '@diegxdelrey is this on ur computer? mine did it too :('}
{'text': 'wanna go to singapore :('}
{'text': 'I WANT 🍜🍜🍜🍜🍜 :((('}
{'text': 'take me back pls :( http://t.co/baFVb7CXWZ'}
{'text': 'THERES THIS VIBRATING NOISE THAT WOKE ME UP ANS I KEEP HEARING IT AND ITS MAKING ME SAD CUS IM SO TIRED :(('}
{'text': '@Charlo_Parker Sorry :('}
{'text': 'So many people are dying while returning to Dhaka after celebrating Eid. :(\n\nGuess traffic jam is in fact a... http://t.co/lR3JUxPtJG'}
{'text': '@Kellipage17 Haha, aw :( Sucks to be you! I shall be there in spirit keeping you warm &amp; cheering for you?'}
{'text': "@Uber How do we get our free Cornettos? We can't figure it out :-( x"}
{'text': '@rainbowsessed nosebleed :('}
{'text': "I had a scary dream &amp; now I can't go back to sleep

In [6]:
def do_filtering( filename_in, filename_out ):
    #filename_out = filename_in[:-7]+'_filtered.txt.gz'
    print('Input:', filename_in, '\t', 'Output:', filename_out)
    file_out = gzip.open(filename_out, mode='wb')
    i = 0
    j = 0
    with io.TextIOWrapper( io.BufferedReader(gzip.open(filename_in)) ) as file_in:
        for line in file_in: # read one line at a time from a compressed file
            json_parsed = json.loads(line) # parse only that one line to json
            lang_bool = English_language_filtering(json_parsed) # True: English, False: not English
            retweet_bool = 'retweeted_status' not in list(json_parsed.keys()) # True: not retweet, False: retweet
            if lang_bool == True and retweet_bool: # if conditions are met write to file
                file_out.write( json.dumps( {'text': json_parsed['text']} ).encode() ) # write dict to file
                file_out.write( '\n'.encode() ) # write a newline char to the end
                #print(i, json_parsed['text'], '\n')
                j += 1
            i += 1
    file_in.close()
    file_out.close()
    print('Number of rows:', i, 'after filtering for language and retweet:', j)

In [7]:
%%time
do_filtering( '../Data/negative_tweets.txt.gz', '../Data/negative_tweets_filtered.txt.gz' )
do_filtering( '../Data/positive_tweets.txt.gz', '../Data/positive_tweets_filtered.txt.gz' )
do_filtering( '../Data/tweets.20150430-223406.txt.gz', '../Data/tweets.20150430-223406_filtered.txt.gz' )

Input: ../Data/negative_tweets.txt.gz 	 Output: ../Data/negative_tweets_filtered.txt.gz
Number of rows: 5000 after filtering for language and retweet: 5000
Input: ../Data/positive_tweets.txt.gz 	 Output: ../Data/positive_tweets_filtered.txt.gz
Number of rows: 5000 after filtering for language and retweet: 5000
Input: ../Data/tweets.20150430-223406.txt.gz 	 Output: ../Data/tweets.20150430-223406_filtered.txt.gz
Number of rows: 20000 after filtering for language and retweet: 6391
CPU times: user 2.36 s, sys: 13.3 ms, total: 2.37 s
Wall time: 2.37 s


### Language processing

In [8]:
from nltk.tokenize import TweetTokenizer
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
import datetime

In [9]:
# Special tokenizer for tweets
tweet_tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)

# Stemmer
snow = SnowballStemmer( 'english', ignore_stopwords=True )
lemma = nltk.wordnet.WordNetLemmatizer()

# Stop words
stop_words = set(stopwords.words('english')) 

In [10]:
def text_tokenizer( text ):
    tokenized_text = []
    
    tokenized = tweet_tokenizer.tokenize(text)
    for item in tokenized:
        match = re.match('#?[A-Za-z]+', item)
        if(match):
            result = match.group(0)
            url_filter = re.match('https?', result)
            #letter_filter = re.match('[A-Za-z]', result) # any other match can be done this way
            if( url_filter == None ):
                real_word = re.findall('\w+', result)[0]
                tokenized_text.append(real_word)
    #print( [ x for x in tokenized_text if ( len(x) > 1 ) ] )
    return [ x for x in tokenized_text if ( len(x) > 1 ) ] # REMOVE words that are only 1 letter long

In [11]:
def text_stemmer( tokenized_text ):
    return [ snow.stem(word) for word in tokenized_text ]

In [12]:
def text_no_stopwords( tokenized_text ):
    return [ w for w in tokenized_text if not w in stop_words ] 

In [13]:
def language_detector(text):
    '''Returns language type as string or 'un' if unknown.'''
    try:
        polyglot = Detector(text, quiet=True).languages[0]
        if polyglot.confidence > 90: # threshold for detecting languages
            lang = str(polyglot.locale)
        else:
            lang = 'un'
    except Exception as e:
        print("Error: " + str(e))
        lang = 'un' # if error occurs during conversion language is unknown
    # print('Guess: ', lang)
    return lang

In [14]:
def English_language_filtering( text ):
    lang = language_detector(text)
    if( lang == 'en' ):
        return True
    else:
        return False

In [15]:
def do_tweet_processing( filename_in, filename_out ):
    print('Input:', filename_in, '\t', 'Output:', filename_out)
    i = 0
    file_out = gzip.open(filename_out, mode='wb')
    acctime = 0

    with io.TextIOWrapper( io.BufferedReader(gzip.open(filename_in)) ) as file_in:
        for line in file_in: # read one line at a time from a compressed file
            t1 = datetime.datetime.now()
            json_parsed = json.loads(line) # parse only that one line to json
            tweet = json_parsed['text'] # get tweet message
            
            tweet_tokenized = text_tokenizer(tweet) # tokenize tweet message
            tweet_stemmed = text_stemmer( tweet_tokenized ) # stem tokenized words
            tweet_filtered = text_no_stopwords( tweet_stemmed ) # remove stop words
            
            lang_bool = English_language_filtering( ' '.join(tweet_tokenized) ) # True: English, False: not English
            
            # check process status
            if i % 1000000 == 0:
                print('status:', i, ', elapsed time:', np.round(acctime*0.001/60/60, 3), 'hour(s)', ', time to finish:', np.round(acctime*(10**8-i)/(i+1)*0.001/60/60, 1), 'hour(s)' )
            if lang_bool == True: # if conditions are met write to file
                file_out.write( ' '.join(tweet_filtered).encode() ) # write to file
                file_out.write( '\n'.encode() ) # write a newline char to the end
            t2 = datetime.datetime.now()
            dt = (t2-t1).microseconds*0.001
            acctime = acctime + dt 
            i += 1
    file_in.close()
    file_out.close()

In [16]:
!ls ../Data/

negative_tweets_filtered.txt.gz
negative_tweets_ready.txt.gz
negative_tweets_ready_withstop.txt.gz
negative_tweets.txt.gz
positive_tweets_filtered.txt.gz
positive_tweets_ready.txt.gz
positive_tweets_ready_withstop.txt.gz
positive_tweets.txt.gz
tweets.20150430-223406_filtered.txt.gz
tweets.20150430-223406_ready.txt.gz
tweets.20150430-223406_ready_withstop.txt.gz
tweets.20150430-223406.txt.gz
twitter_samples.zip
World_twitterdata_part1.txt.gz
World_twitterdata_part2.txt.gz
World_twitterdata_part3.txt.gz
World_twitterdata_part4.txt.gz
World_twitterdata_part5.txt.gz
World_twitterdata_ready_part1.txt.gz
World_twitterdata_ready_part2.txt.gz
World_twitterdata_ready_part3.txt.gz
World_twitterdata_ready_part4.txt.gz
World_twitterdata_ready_part5.txt.gz
World_twitterdata_ready.txt.gz
World_twitterdata.txt.gz


In [17]:
%%time
do_tweet_processing( '../Data/negative_tweets_filtered.txt.gz', '../Data/negative_tweets_ready.txt.gz' )
do_tweet_processing( '../Data/positive_tweets_filtered.txt.gz', '../Data/positive_tweets_ready.txt.gz' )
do_tweet_processing( '../Data/tweets.20150430-223406_filtered.txt.gz', '../Data/tweets.20150430-223406_ready.txt.gz' )

Input: ../Data/negative_tweets_filtered.txt.gz 	 Output: ../Data/negative_tweets_ready.txt.gz
status: 0 , elapsed time: 0.0 hour(s) , time to finish: 0.0 hour(s)
Input: ../Data/positive_tweets_filtered.txt.gz 	 Output: ../Data/positive_tweets_ready.txt.gz
status: 0 , elapsed time: 0.0 hour(s) , time to finish: 0.0 hour(s)
Input: ../Data/tweets.20150430-223406_filtered.txt.gz 	 Output: ../Data/tweets.20150430-223406_ready.txt.gz
status: 0 , elapsed time: 0.0 hour(s) , time to finish: 0.0 hour(s)
CPU times: user 3.44 s, sys: 7.98 ms, total: 3.45 s
Wall time: 3.47 s


### Not filtering stop words

In [18]:
def do_tweet_processing( filename_in, filename_out ):
    print('Input:', filename_in, '\t', 'Output:', filename_out)
    i = 0
    file_out = gzip.open(filename_out, mode='wb')
    acctime = 0

    with io.TextIOWrapper( io.BufferedReader(gzip.open(filename_in)) ) as file_in:
        for line in file_in: # read one line at a time from a compressed file
            t1 = datetime.datetime.now()
            json_parsed = json.loads(line) # parse only that one line to json
            tweet = json_parsed['text'] # get tweet message
            
            tweet_tokenized = text_tokenizer(tweet) # tokenize tweet message
            tweet_stemmed = text_stemmer( tweet_tokenized ) # stem tokenized words
            #tweet_filtered = text_no_stopwords( tweet_stemmed ) # remove stop words
            
            lang_bool = English_language_filtering( ' '.join(tweet_tokenized) ) # True: English, False: not English
            
            # check process status
            if i % 1000000 == 0:
                print('status:', i, ', elapsed time:', np.round(acctime*0.001/60/60, 3), 'hour(s)', ', time to finish:', np.round(acctime*(10**8-i)/(i+1)*0.001/60/60, 1), 'hour(s)' )
            if lang_bool == True: # if conditions are met write to file
                file_out.write( ' '.join(tweet_stemmed).encode() ) # write to file
                file_out.write( '\n'.encode() ) # write a newline char to the end
            t2 = datetime.datetime.now()
            dt = (t2-t1).microseconds*0.001
            acctime = acctime + dt 
            i += 1
    file_in.close()
    file_out.close()

In [19]:
%%time
do_tweet_processing( '../Data/negative_tweets_filtered.txt.gz', '../Data/negative_tweets_ready_withstop.txt.gz' )
do_tweet_processing( '../Data/positive_tweets_filtered.txt.gz', '../Data/positive_tweets_ready_withstop.txt.gz' )
do_tweet_processing( '../Data/tweets.20150430-223406_filtered.txt.gz', '../Data/tweets.20150430-223406_ready_withstop.txt.gz' )

Input: ../Data/negative_tweets_filtered.txt.gz 	 Output: ../Data/negative_tweets_ready_withstop.txt.gz
status: 0 , elapsed time: 0.0 hour(s) , time to finish: 0.0 hour(s)
Input: ../Data/positive_tweets_filtered.txt.gz 	 Output: ../Data/positive_tweets_ready_withstop.txt.gz
status: 0 , elapsed time: 0.0 hour(s) , time to finish: 0.0 hour(s)
Input: ../Data/tweets.20150430-223406_filtered.txt.gz 	 Output: ../Data/tweets.20150430-223406_ready_withstop.txt.gz
status: 0 , elapsed time: 0.0 hour(s) , time to finish: 0.0 hour(s)
CPU times: user 3.5 s, sys: 0 ns, total: 3.5 s
Wall time: 3.51 s


### Notes:
* After removing retweets, only 1/3 of the tweets are kept in the 20000 long file!